# Transfer Learning

In [1]:
import pandas as pd
import numpy as np
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D,concatenate,\
Dense,Dropout,LSTM,Masking,Bidirectional,Dropout,GRU,SimpleRNN,TimeDistributed
from keras.models import Sequential,Model
import tensorflow as tf
from sklearn.metrics import accuracy_score,precision_recall_curve,roc_auc_score,auc
import collections
from keras import backend as K



path='/gdrive/My Drive/Colab Notebooks/ML4HC/project3/Data/'

df_1 = pd.read_csv(path+"ptbdb_normal.csv", header=None)
df_2 = pd.read_csv(path+"ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [0]:
def auc_score(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

def AUPRC(y_true,y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    return auc(recall, precision)

#### All the models including in this section can be loaded from the external file called Models. The 2 stacked Bidirectional GRU is used as the base model.

In [ ]:
path='/gdrive/My Drive/Colab Notebooks/ML4HC/project3/Models/'

## Freeze Initial model, train only the final layer


In [0]:
model=Sequential()
model.add(Masking(mask_value=0.,input_shape=(187,1)))
model.add(Bidirectional(GRU(100,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(100)))
model.add(Dropout(0.2))
model.add(Dense(5,activation=activations.softmax))

model.load_weights(path+'2_Bid_GRU_100_100dropout0202.h5')
model.summary()

In [0]:
#Delete the output layer
model.pop()

In [0]:
#Freeze the model
model.layers[1].trainable=False
model.layers[3].trainable=False

# Add trainable output layer
model.add(Dense(1,activation=activations.sigmoid))

In [0]:
opt = optimizers.Adam(0.001)

model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc',auc_score])
model.summary()

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 187, 1)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 187, 200)          61200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 187, 200)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               180600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 24

In [0]:
file_path = path+"Transfer_one.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

#model.fit(X, Y, epochs=100, verbose=2, callbacks=callbacks_list, validation_split=0.1)

In [0]:
model.load_weights(path+"Transfer_one.h5")

pred_test = model.predict(X_test)
print("Test AUPRC score : %s "% AUPRC(Y_test,pred_test))
print("Test AUROC score : %s "% roc_auc_score(Y_test,pred_test))

pred_test = (pred_test>0.5).astype(np.int8)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test AUPRC score : 0.9704612178250035 
Test AUROC score : 0.9289063685300596 
Test accuracy score : 0.8763311576777739 


## Retrain the whole model

In [0]:
model=Sequential()
model.add(Masking(mask_value=0.,input_shape=(187,1)))
model.add(Bidirectional(GRU(100,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(100)))
model.add(Dropout(0.2))
model.add(Dense(5,activation=activations.softmax))

model.load_weights(path+'2_Bid_GRU_100_100dropout0202.h5')

model.pop()

model.add(Dense(1,activation=activations.sigmoid))

model.summary()

opt = optimizers.Adam(0.001)

model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc',auc_score])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, 187, 1)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 187, 200)          61200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 187, 200)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               180600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
Total params: 242,001
Trainable params: 242,001
Non-trainable params: 0
_________________________________________________________________


In [0]:
file_path =path+ "Transfer_two.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

#model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)

In [0]:
model.load_weights(path+"Transfer_two.h5")

pred_test = model.predict(X_test)
print("Test AUPRC score : %s "% AUPRC(Y_test,pred_test))
print("Test AUROC score : %s "% roc_auc_score(Y_test,pred_test))

pred_test = (pred_test>0.5).astype(np.int8)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test AUPRC score : 0.9995184606425771 
Test AUROC score : 0.9988791650544129 
Test accuracy score : 0.9931295087598764 


## First Freeze ,then retrain the whole model

In [0]:
model=Sequential()
model.add(Masking(mask_value=0.,input_shape=(187,1)))
model.add(Bidirectional(GRU(100,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(100)))
model.add(Dropout(0.2))
model.add(Dense(1,activation=activations.sigmoid))

#load the model from transfer learning part 1
model.load_weights(path+'Transfer_one.h5')


model.summary()

opt = optimizers.Adam(0.001)

model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc',auc_score])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_3 (Masking)          (None, 187, 1)            0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 187, 200)          61200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 187, 200)          0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200)               180600    
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
Total params: 242,001
Trainable params: 242,001
Non-trainable params: 0
_________________________________________________________________


In [0]:
file_path =path+ "Transfer_three.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

#model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)

In [0]:
model.load_weights(path+"Transfer_three.h5")

pred_test = model.predict(X_test)
print("Test AUPRC score : %s "% AUPRC(Y_test,pred_test))
print("Test AUROC score : %s "% roc_auc_score(Y_test,pred_test))

pred_test = (pred_test>0.5).astype(np.int8)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test AUPRC score : 0.9994456356954413 
Test AUROC score : 0.9988876918680072 
Test accuracy score : 0.9917554105118516 


# Results

In [14]:
models=['Train only the final layer','Retrain the whole model','First Freeze,then retrain the whole model']
Test_acc=[0.876,0.9931,0.9917]
Test_AUROC=[0.929,0.9989,0.9989]
Test_AUPRC=[0.971,0.999,0.999]
Results=pd.DataFrame(index=models)
Results['Test Accuracy']=Test_acc
Results['Test AUROC']=Test_AUROC
Results['Test_AUPRC']=Test_AUPRC
Results

,Test Accuracy,Test AUROC,Test_AUPRC
Train only the final layer,0.8760,0.9290,0.971
Retrain the whole model,0.9931,0.9989,0.999
"First Freeze,then retrain the whole model",0.9917,0.9989,0.999
